In [115]:
def discretize_bearing(bearing):
  """
  방위각(0~360도)을 0, 90, 180, 270도 중 가장 가까운 값으로 정제합니다.
  """
  # 북쪽 기준을 위해 22.5도를 더하고, 360도로 나눈 나머지를 취합니다.
  # 이렇게 하면 북쪽(0도)을 기준으로 22.5~67.5도는 45도 범위에 들어갑니다.
  adjusted_bearing = (bearing + 45) % 360 

  # adjusted_bearing을 90으로 나누어 몇 번째 사분면에 가까운지 확인
  # 예: 10도 -> (10+45)%360 = 55 -> 55/90 = 0.61 -> round(0.61) * 90 = 0
  # 예: 100도 -> (100+45)%360 = 145 -> 145/90 = 1.61 -> round(1.61) * 90 = 180 (X) -> 이 방법은 복잡하고 오류 가능성이 높습니다.

  # 직관적인 방법을 사용합니다.
  
  # 45도로 나누어 몫을 구합니다.
  # 0~45도: 0/90 (북)
  # 45~135도: 90/90 (동)
  # 135~225도: 180/90 (남)
  # 225~315도: 270/90 (서)
  # 315~360도: 360/90 (북)
  
  # 북쪽을 $360^\circ$로 생각하고 조정하여 반올림합니다.
  if bearing > 315 or bearing <= 45:
    return 0 # 북
  elif bearing > 45 and bearing <= 135:
    return 90 # 동
  elif bearing > 135 and bearing <= 225:
    return 180 # 남
  elif bearing > 225 and bearing <= 315:
    return 270 # 서
  else:
    # 이 부분은 발생하지 않아야 하지만, 안전을 위해 0을 반환합니다.
    return 0 


In [116]:


# 예시 테스트:
print(f"방위각 20도 -> 정제 방향: {discretize_bearing(20)}도 (북)")
print(f"방위각 110도 -> 정제 방향: {discretize_bearing(110)}도 (동)")
print(f"방위각 200도 -> 정제 방향: {discretize_bearing(200)}도 (남)")
print(f"방위각 300도 -> 정제 방향: {discretize_bearing(300)}도 (서)")
print(f"방위각 350도 -> 정제 방향: {discretize_bearing(350)}도 (북)")

방위각 20도 -> 정제 방향: 0도 (북)
방위각 110도 -> 정제 방향: 90도 (동)
방위각 200도 -> 정제 방향: 180도 (남)
방위각 300도 -> 정제 방향: 270도 (서)
방위각 350도 -> 정제 방향: 0도 (북)


In [117]:
import math

def calculate_bearing(lat1, lon1, lat2, lon2):
  # 1. 모든 도(Degree) 값을 라디안(Radian)으로 변환
  lat1_rad = math.radians(float(lat1))
  lon1_rad = math.radians(float(lon1))
  lat2_rad = math.radians(float(lat2))
  lon2_rad = math.radians(float(lon2))

  # 경도 차이
  dLon = lon2_rad - lon1_rad

  # 2. 방위각 계산 공식 적용 (arctan2 사용)
  y = math.sin(dLon) * math.cos(lat2_rad)
  x = math.cos(lat1_rad) * math.sin(lat2_rad) - math.sin(lat1_rad) * math.cos(lat2_rad) * math.cos(dLon)
  
  # atan2는 라디안으로 결과를 반환하며, x, y 값을 통해 사분면을 고려함
  bearing_rad = math.atan2(y, x)

  # 3. 라디안을 도(Degree)로 변환
  bearing_deg = math.degrees(bearing_rad)

  # 4. 0 ~ 360도 범위로 조정
  # 결과가 음수일 경우 360을 더해서 양수로 만듦 (예: -90 -> 270)
  final_bearing = (bearing_deg + 360) % 360

  return final_bearing


In [118]:

# 예시: 서울(출발지)에서 부산(도착지) 방향
seoul_lat, seoul_lon = 37.5665, 126.9780
busan_lat, busan_lon = 35.1796, 129.0756

direction = calculate_bearing(seoul_lat, seoul_lon, busan_lat, busan_lon)

print(f"서울에서 부산으로의 이동 방향 (방위각): {direction:.2f}도, 정제{discretize_bearing(direction)}") 
# 결과는 약 154.55도로, 남동쪽 (135도) 방향에 가까움을 알 수 있습니다.

서울에서 부산으로의 이동 방향 (방위각): 144.09도, 정제180


In [119]:
def calcAngle(originX, originY, destX, destY):
    return discretize_bearing(calculate_bearing(originY, originX, destY, destX))

In [120]:
print("calcAngle=", calcAngle(seoul_lon, seoul_lat, busan_lon, busan_lat))

calcAngle= 180


In [121]:
import requests

# 2개스페이스로 된 탭으로 보여줍니다.
# URL 및 헤더 정보
url = "https://apis-navi.kakaomobility.com/v1/directions"
headers = {
  "Authorization": "KakaoAK 430d403638a9ccddb70657adb3ed7196"
}

originX = "127.1027997"
originY = "37.61354475"
destX = "127.099763"
destY = "37.614242"

# 쿼리 파라미터 (Query Parameters)
# curl 명령어의 '?' 뒤에 오는 부분입니다.
params = {
  "origin": f"{originX},{originY},angle={calcAngle(originX, originY, destX, destY)}",
  "destination": f"{destX},{destY}",
  "priority": "DISTANCE", # DISTANCE:최단거리, TIME:최단시간, RECOMMEND:추천
  "car_type": "3", # 1:소형,2:중형,3:대형(33인승이상)
  "summary":"true" #요약정보만
}

try:
  # GET 요청 보내기
  response = requests.get(url, headers=headers, params=params)

  # 응답 확인
  if response.status_code == 200:
    # 성공 시 JSON 응답 출력
    print("요청 성공 (Status Code: 200)")
    # print(response.json()) # 전체 JSON 응답을 보고 싶을 때
    
    # 예시로 응답의 주요 정보만 출력
    data = response.json()
    if 'routes' in data and data['routes']:
      print(f"경로 수: {len(data['routes'])}개")
      first_route = data['routes'][0]['summary']
      print(f"총 거리: {first_route['distance']} m")
      print(f"총 시간: {first_route['duration']} 초")
    else:
      print("경로 데이터가 응답에 포함되어 있지 않습니다.")
    
  else:
    # 오류 발생 시 상태 코드 및 내용 출력
    print(f"요청 실패 (Status Code: {response.status_code})")
    print(f"오류 메시지: {response.text}")

except requests.exceptions.RequestException as e:
  # 네트워크 등 요청 자체의 오류 처리
  print(f"요청 중 오류 발생: {e}")

요청 성공 (Status Code: 200)
경로 수: 1개
총 거리: 278 m
총 시간: 80 초


In [123]:
import pandas as pd
import numpy as np

src_df = pd.read_csv('20251014_서울시_버스정류장_주소.csv')

In [124]:
src_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41261 entries, 0 to 41260
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ROUTE_ID  41261 non-null  int64  
 1   노선명       41261 non-null  object 
 2   순번        41261 non-null  int64  
 3   NODE_ID   41261 non-null  int64  
 4   ARS_ID    41261 non-null  int64  
 5   정류소명      41261 non-null  object 
 6   X좌표       41261 non-null  float64
 7   Y좌표       41261 non-null  float64
 8   시         41261 non-null  object 
 9   구         41261 non-null  object 
 10  동         41261 non-null  object 
dtypes: float64(2), int64(4), object(5)
memory usage: 3.5+ MB


In [125]:
src_df['거리'] = 0

In [127]:
src_df

,ROUTE_ID,노선명,순번,NODE_ID,ARS_ID,정류소명,X좌표,Y좌표,시,구,동,거리
0,100100407,240,1,106000319,7418,중랑공영차고지.신내역,127.102800,37.613545,서울특별시,중랑구,신내동,0
1,100100049,273,1,106000319,7418,중랑공영차고지.신내역,127.102800,37.613545,서울특별시,중랑구,신내동,0
2,100100186,2012,1,106000319,7418,중랑공영차고지.신내역,127.102800,37.613545,서울특별시,중랑구,신내동,0
3,100100190,2015,1,106000319,7418,중랑공영차고지.신내역,127.102800,37.613545,서울특별시,중랑구,신내동,0
4,100100599,2311,1,106000319,7418,중랑공영차고지.신내역,127.102800,37.613545,서울특별시,중랑구,신내동,0
...,...,...,...,...,...,...,...,...,...,...,...,...
41256,100100589,N61,183,116000142,17233,개봉역,126.856720,37.493978,서울특별시,구로구,개봉동,0
41257,100100589,N61,184,116000144,17235,개봉1동사거리.개봉푸르지오아파트,126.846791,37.501413,서울특별시,구로구,개봉동,0
41258,100100589,N61,185,114000125,15228,신정학마을아파트,126.843963,37.507421,서울특별시,양천구,신정동,0
41259,100100589,N61,186,114000257,15362,신정현대아파트.서부트럭터미널,126.842007,37.509011,서울특별시,양천구,신정동,0


In [128]:
import pandas as pd
import requests
import time
import json
from io import StringIO
from math import isnan # NaN 값 비교를 위해 임포트

In [129]:
grouped = src_df.groupby('노선명')

In [131]:
print(len(grouped))

705


In [133]:
# grouped 객체의 반복자에서 첫 번째 요소를 가져옵니다.
first_element = next(iter(grouped))

# first_element는 (그룹 이름, 그룹 DataFrame) 형태의 튜플입니다.
# 이를 두 변수로 나누어 받을 수 있습니다.
group_name, group_df = first_element

# 첫 번째 그룹 이름 출력
print(f"첫 번째 그룹 이름: {group_name}")

# 첫 번째 그룹의 데이터프레임 출력
print("--- 첫 번째 그룹 데이터프레임 ---")
print(group_df)

첫 번째 그룹 이름: 0017
--- 첫 번째 그룹 데이터프레임 ---
        ROUTE_ID   노선명  순번    NODE_ID  ARS_ID             정류소명         X좌표  \
720    100100124  0017   2  102000158    3252          신용산지하차도  126.963882   
1663   100100124  0017   3  102000161    3255              용산역  126.965822   
2683   100100124  0017   4  102000162    3256         용산푸르지오써밋  126.964759   
3254   100100124  0017   5  102000029    3122           한강대교북단  126.963570   
3534   100100124  0017   6  102000200    3294          서부이촌동입구  126.959497   
4655   100100124  0017   7  102000198    3292  이촌2동대림아파트.새남터성지  126.956323   
5258   100100124  0017   8  102000196    3290         이촌2동주민센터  126.955064   
5945   100100124  0017   9  102000194    3288     성촌공원.포르쉐센터용산  126.954606   
6671   100100124  0017  10  102000193    3287   이촌119안전센터.성촌공원  126.952943   
7337   100100124  0017  11  102000202    3296      원효2동산호아파트후문  126.950016   
7787   100100124  0017  12  102000204    3298       청암동강변삼성아파트  126.949323   
8671   100100124  0017  

In [134]:
route_each_df = src_df[src_df['노선명'] == '0017'].copy()
route_each_df

,ROUTE_ID,노선명,순번,NODE_ID,ARS_ID,정류소명,X좌표,Y좌표,시,구,동,거리
720,100100124,0017,2,102000158,3252,신용산지하차도,126.963882,37.532590,서울특별시,용산구,한강로3가,0
1663,100100124,0017,3,102000161,3255,용산역,126.965822,37.528869,서울특별시,용산구,한강로3가,0
2683,100100124,0017,4,102000162,3256,용산푸르지오써밋,126.964759,37.527574,서울특별시,용산구,한강로3가,0
3254,100100124,0017,5,102000029,3122,한강대교북단,126.963570,37.524968,서울특별시,용산구,한강로3가,0
3534,100100124,0017,6,102000200,3294,서부이촌동입구,126.959497,37.523863,서울특별시,용산구,한강로3가,0
4655,100100124,0017,7,102000198,3292,이촌2동대림아파트.새남터성지,126.956323,37.526175,서울특별시,용산구,이촌동,0
5258,100100124,0017,8,102000196,3290,이촌2동주민센터,126.955064,37.527736,서울특별시,용산구,한강로3가,0
5945,100100124,0017,9,102000194,3288,성촌공원.포르쉐센터용산,126.954606,37.530171,서울특별시,용산구,한강로3가,0
6671,100100124,0017,10,102000193,3287,이촌119안전센터.성촌공원,126.952943,37.530082,서울특별시,용산구,이촌동,0
7337,100100124,0017,11,102000202,3296,원효2동산호아파트후문,126.950016,37.532260,서울특별시,용산구,원효로4가,0


In [135]:
import requests
import numpy as np

def calcDistanceByApi(originX, originY, destX, destY):
    # 2개스페이스로 된 탭으로 보여줍니다.
    # URL 및 헤더 정보
    url = "https://apis-navi.kakaomobility.com/v1/directions"
    headers = {
        "Authorization": "KakaoAK 2a96ad655ffebec848633bc4779b2bf6"
    }

    # 쿼리 파라미터 (Query Parameters)
    # curl 명령어의 '?' 뒤에 오는 부분입니다.
    params = {
        "origin": f"{originX},{originY},angle={calcAngle(originX, originY, destX, destY)}",
        "destination": f"{destX},{destY}",
        "priority": "DISTANCE", # DISTANCE:최단거리, TIME:최단시간, RECOMMEND:추천
        "car_type": "3", # 1:소형,2:중형,3:대형(33인승이상)
        "summary":"true", #요약정보만,
        "avoid":"ferries|uturn",
        "alternatives":"true",
        "roadevent":"2"
    }

    try:
        # GET 요청 보내기
        response = requests.get(url, headers=headers, params=params)

        # 응답 확인
        if response.status_code == 200:
            # 성공 시 JSON 응답 출력
            #print("요청 성공 (Status Code: 200)")
            # print(response.json()) # 전체 JSON 응답을 보고 싶을 때
            
            # 예시로 응답의 주요 정보만 출력
            data = response.json()
            
            # 🌟 수정 부분: 'routes' 키는 확인되었으나, 'summary' 키가 없을 경우 처리
            if 'routes' in data and data['routes']:
                first_route = data['routes'][0]
                
                # 'routes' 리스트의 첫 번째 요소가 예상대로 'summary'를 가지고 있는지 확인
                try:
                    return first_route['summary']['distance']
                
                except KeyError as e:
                    # 1. 'result_code'가 104인지 확인
                    if 'result_code' in first_route and first_route['result_code'] == 104:
                        # result_code 104: "출발지와 도착지가 5 m 이내"
                        # 요청에 따라 5m를 반환합니다.
                        return 5.0 # 거리 단위는 미터(m)

                    # 2. 기타 KeyError 처리 및 디버깅 출력 (이전과 동일)
                    print("-" * 50)
                    print(f"KeyError 발생: {e} 키가 첫 번째 경로 객체에 없습니다.")
                    print("오류 발생 시의 응답 데이터 (data) 전체:")
                    print(data) 
                    print("-" * 50)
                    return np.nan 
                
            else:
                # 'routes' 키가 없거나 리스트가 비어있을 경우 (경로 탐색 실패 등)
                print("경로 데이터가 응답에 포함되어 있지 않거나 비어있습니다.")
                print("응답 데이터 (data) 전체:")
                print(data)
                return np.nan
            
        else:
            # 오류 발생 시 상태 코드 및 내용 출력
            print(f"요청 실패 (Status Code: {response.status_code})")
            print(f"오류 메시지: {response.text}")

    except requests.exceptions.RequestException as e:
        # 네트워크 등 요청 자체의 오류 처리
        print(f"요청 중 오류 발생: {e}=data{data}")

In [136]:
print("aaa=",calcDistanceByApi(originX, originY, destX, destY))

aaa= 278


In [137]:
DISTANCE_CACHE = []

In [138]:
import time
from tqdm import tqdm
import numpy as ny

def calculate_route_distances_pseudo(df):
  """
  인덱스 매핑 오류를 수정한 함수입니다. 원본 인덱스를 저장하여 정확히 매핑합니다.
  """
  # 결과를 담을 새로운 컬럼 초기화
  df['거리'] = 0.0

  # '노선명'을 기준으로 그룹화
  grouped = df.groupby('노선명')

  # 각 노선별로 순회하며 거리 계산
  for route_name, route_df in tqdm(grouped, desc="노선별 거리 계산 진행", total=len(grouped)):
    # 🌟 수정 포인트 1: 원래의 데이터프레임 인덱스를 저장합니다.
    original_indices = route_df.index
    
    # 순번 오름차순으로 정렬 후 인덱스 재설정 (계산 편의를 위해 0, 1, 2...로 만듦)
    route_df = route_df.sort_values(by='순번').reset_index(drop=True)
    
    route_distances = []
    
    # 거리 계산 루프 (N-1)
    for i in range(len(route_df) - 1):
      lat1 = route_df.loc[i, 'Y좌표']
      lon1 = route_df.loc[i, 'X좌표']
      lat2 = route_df.loc[i + 1, 'Y좌표']
      lon2 = route_df.loc[i + 1, 'X좌표']

      cache_key = (lat1, lon1, lat2, lon2)

      print(f"노선={route_df.loc[i, '노선명']},순번={route_df.loc[i, '순번']},key={cache_key}")
        


In [139]:
calculate_route_distances_pseudo(src_df)

노선별 거리 계산 진행:  12%|███████████▊                                                                                      | 85/705 [00:00<00:01, 446.71it/s]

노선=0017,순번=2,key=(np.float64(37.53259), np.float64(126.963882), np.float64(37.528869), np.float64(126.965822))
노선=0017,순번=3,key=(np.float64(37.528869), np.float64(126.965822), np.float64(37.527574), np.float64(126.964759))
노선=0017,순번=4,key=(np.float64(37.527574), np.float64(126.964759), np.float64(37.524968), np.float64(126.96357))
노선=0017,순번=5,key=(np.float64(37.524968), np.float64(126.96357), np.float64(37.523863), np.float64(126.959497))
노선=0017,순번=6,key=(np.float64(37.523863), np.float64(126.959497), np.float64(37.52617531), np.float64(126.9563231))
노선=0017,순번=7,key=(np.float64(37.52617531), np.float64(126.9563231), np.float64(37.52773644), np.float64(126.955064))
노선=0017,순번=8,key=(np.float64(37.52773644), np.float64(126.955064), np.float64(37.53017103), np.float64(126.9546065))
노선=0017,순번=9,key=(np.float64(37.53017103), np.float64(126.9546065), np.float64(37.530082), np.float64(126.952943))
노선=0017,순번=10,key=(np.float64(37.530082), np.float64(126.952943), np.float64(37.53226), np.

노선별 거리 계산 진행:  41%|███████████████████████████████████████▎                                                         | 286/705 [00:00<00:01, 409.33it/s]

노선=3322,순번=1,key=(np.float64(37.46992977), np.float64(127.1275898), np.float64(37.47903951), np.float64(127.1261442))
노선=3322,순번=2,key=(np.float64(37.47903951), np.float64(127.1261442), np.float64(37.482125), np.float64(127.124748))
노선=3322,순번=3,key=(np.float64(37.482125), np.float64(127.124748), np.float64(37.48612299), np.float64(127.1246073))
노선=3322,순번=4,key=(np.float64(37.48612299), np.float64(127.1246073), np.float64(37.486465), np.float64(127.12834))
노선=3322,순번=5,key=(np.float64(37.486465), np.float64(127.12834), np.float64(37.48892393), np.float64(127.1319278))
노선=3322,순번=6,key=(np.float64(37.48892393), np.float64(127.1319278), np.float64(37.49122303), np.float64(127.1345693))
노선=3322,순번=7,key=(np.float64(37.49122303), np.float64(127.1345693), np.float64(37.49238684), np.float64(127.1359636))
노선=3322,순번=8,key=(np.float64(37.49238684), np.float64(127.1359636), np.float64(37.49420734), np.float64(127.1380636))
노선=3322,순번=9,key=(np.float64(37.49420734), np.float64(127.1380636), np

노선별 거리 계산 진행:  58%|████████████████████████████████████████████████████████▏                                        | 408/705 [00:00<00:00, 508.77it/s]IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [142]:
src_df = src_df.sort_values(by=['노선명', '순번'])

In [143]:
src_df

,ROUTE_ID,노선명,순번,NODE_ID,ARS_ID,정류소명,X좌표,Y좌표,시,구,동,거리
720,100100124,0017,2,102000158,3252,신용산지하차도,126.963882,37.532590,서울특별시,용산구,한강로3가,0.0
1663,100100124,0017,3,102000161,3255,용산역,126.965822,37.528869,서울특별시,용산구,한강로3가,0.0
2683,100100124,0017,4,102000162,3256,용산푸르지오써밋,126.964759,37.527574,서울특별시,용산구,한강로3가,0.0
3254,100100124,0017,5,102000029,3122,한강대교북단,126.963570,37.524968,서울특별시,용산구,한강로3가,0.0
3534,100100124,0017,6,102000200,3294,서부이촌동입구,126.959497,37.523863,서울특별시,용산구,한강로3가,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
832,100000020,청와대A01,2,100000415,1119,영추문,126.973982,37.578930,서울특별시,종로구,세종로,0.0
2061,100000020,청와대A01,3,100000416,1601,청와대,126.973762,37.582971,서울특별시,종로구,세종로,0.0
2767,100000020,청와대A01,4,100000417,1602,춘추문,126.979657,37.583126,서울특별시,종로구,팔판동,0.0
3474,100000020,청와대A01,5,100000418,1603,경복궁.국립민속박물관,126.979590,37.579407,서울특별시,종로구,세종로,0.0


In [242]:
import pickle

def load_cache_from_pickle(filename):
  try:
    with open(filename, 'rb') as f: # 'rb'는 Read Binary 모드
      loaded_cache = pickle.load(f)
    print(f"캐시를 '{filename}'에서 성공적으로 불러왔습니다.")
    return loaded_cache
  except FileNotFoundError:
    print(f"경고: '{filename}' 파일을 찾을 수 없습니다. 빈 캐시를 반환합니다.")
    return []

In [ ]:
DISTANCE_CACHE_LOADED = load_cache_from_pickle("20251027_DISTANCE_CACHE.pkl") 

캐시를 '20251015_DISTANCE_CACHE.pkl'에서 성공적으로 불러왔습니다.


In [ ]:
print(f"len={len(DISTANCE_CACHE_LOADED)}")

len17229


In [157]:
echo_route_df = src_df[src_df['노선명'] == '0017']

In [158]:
echo_route_df

,ROUTE_ID,노선명,순번,NODE_ID,ARS_ID,정류소명,X좌표,Y좌표,시,구,동,거리
720,100100124,0017,2,102000158,3252,신용산지하차도,126.963882,37.532590,서울특별시,용산구,한강로3가,0.0
1663,100100124,0017,3,102000161,3255,용산역,126.965822,37.528869,서울특별시,용산구,한강로3가,0.0
2683,100100124,0017,4,102000162,3256,용산푸르지오써밋,126.964759,37.527574,서울특별시,용산구,한강로3가,0.0
3254,100100124,0017,5,102000029,3122,한강대교북단,126.963570,37.524968,서울특별시,용산구,한강로3가,0.0
3534,100100124,0017,6,102000200,3294,서부이촌동입구,126.959497,37.523863,서울특별시,용산구,한강로3가,0.0
4655,100100124,0017,7,102000198,3292,이촌2동대림아파트.새남터성지,126.956323,37.526175,서울특별시,용산구,이촌동,0.0
5258,100100124,0017,8,102000196,3290,이촌2동주민센터,126.955064,37.527736,서울특별시,용산구,한강로3가,0.0
5945,100100124,0017,9,102000194,3288,성촌공원.포르쉐센터용산,126.954606,37.530171,서울특별시,용산구,한강로3가,0.0
6671,100100124,0017,10,102000193,3287,이촌119안전센터.성촌공원,126.952943,37.530082,서울특별시,용산구,이촌동,0.0
7337,100100124,0017,11,102000202,3296,원효2동산호아파트후문,126.950016,37.532260,서울특별시,용산구,원효로4가,0.0


In [195]:
bus_list = []

In [196]:
def check_in_cache(cached, bus_no):
    org_df = globals()['src_df']

    each_route_df = org_df[org_df['노선명'] == bus_no]

    original_indices = each_route_df.index

    for i in range(len(original_indices) - 1):
        idx1 = original_indices[i]
        idx2 = original_indices[i+1]    

        lat1 = org_df.loc[idx1, 'Y좌표']
        lon1 = org_df.loc[idx1, 'X좌표']
        lat2 = org_df.loc[idx2, 'Y좌표']
        lon2 = org_df.loc[idx2, 'X좌표']

        cache_key = (lat1, lon1, lat2, lon2)
        
        if cache_key in cached:
            distance = cached[cache_key]
            #print("cache hit!!", distance)
            
            org_df.loc[idx1, '거리'] = distance
        else:
            print(f"bus_no={bus_no},not in cache idx={i}")
            globals()['bus_list'].append(bus_no)
            return

    #print(org_df[org_df['노선명'] == bus_no])

In [197]:
grouped = src_df.groupby('노선명')

  # 각 노선별로 순회하며 거리 계산
for route_name, route_df in tqdm(grouped, desc="노선별 거리 계산 진행", total=len(grouped)):
    #print(f"route_name={route_name}")
    check_in_cache(DISTANCE_CACHE_LOADED, route_name)

노선별 거리 계산 진행:   0%|                                                                                                            | 0/705 [00:00<?, ?it/s]

노선별 거리 계산 진행:  17%|████████████████▎                                                                                | 119/705 [00:00<00:04, 123.53it/s]

bus_no=2412,not in cache idx=47


노선별 거리 계산 진행:  23%|██████████████████████▌                                                                          | 164/705 [00:01<00:03, 138.74it/s]

bus_no=3416,not in cache idx=39
bus_no=3426,not in cache idx=12
bus_no=361,not in cache idx=9
bus_no=402,not in cache idx=11
bus_no=4425,not in cache idx=38


...

노선별 거리 계산 진행:  65%|██████████████████████████████████████████████████████████████▉                                  | 457/705 [00:03<00:01, 176.05it/s]

bus_no=N6703,not in cache idx=33
bus_no=강남03,not in cache idx=22
bus_no=강남06,not in cache idx=10
bus_no=강남06-1,not in cache idx=9
bus_no=강남06-2,not in cache idx=9


노선별 거리 계산 진행:  92%|████████████████████████████████████████████████████████████████████████████████████████▉        | 646/705 [00:03<00:00, 299.73it/s]

bus_no=서초06,not in cache idx=4
bus_no=서초16,not in cache idx=16
bus_no=서초17,not in cache idx=36


노선별 거리 계산 진행: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 705/705 [00:03<00:00, 178.42it/s]


In [198]:
bus_list

['2412',
 '3416',
 '3426',
 '361',
 '402',
 '4425',
 '500',
 '504',
 '5525',
 '5535',
 '6001',
 '6005',
 '6006',
 '6007',
 '6008',
 '6009',
 '6013',
 '6016',
 '6017',
 '6019',
 '6020',
 '6103',
 '6200',
 '6300',
 '6600',
 '6703',
 '6705A',
 'N6703',
 '강남03',
 '강남06',
 '강남06-1',
 '강남06-2',
 '서초06',
 '서초16',
 '서초17']

In [190]:
print(f"'특정컬럼'의 값이 0.0인 행의 개수: {(src_df['거리'] == 0.0).sum()}")

'특정컬럼'의 값이 0.0인 행의 개수: 1875


In [ ]:
src_df[src_df['노선명'] == '3416']

,ROUTE_ID,노선명,순번,NODE_ID,ARS_ID,정류소명,X좌표,Y좌표,시,구,동,거리
66,100100228,3416,1,123000337,24429,남한산성입구,127.157278,37.490960,서울특별시,송파구,마천동,115.0
1038,100100228,3416,2,123000200,24290,송파구립마천청소년센터,127.156790,37.491832,서울특별시,송파구,마천동,551.0
1841,100100228,3416,3,123000201,24291,송파보훈회관.금호어울림2차,127.155809,37.493968,서울특별시,송파구,마천동,396.0
2560,100100228,3416,4,123000203,24293,마천동금호어울림1차아파트,127.154198,37.497198,서울특별시,송파구,마천동,434.0
3237,100100228,3416,5,123000206,24296,남천초교.마천동우방아파트앞,127.151931,37.499919,서울특별시,송파구,마천동,209.0
...,...,...,...,...,...,...,...,...,...,...,...,...
31250,100100228,3416,61,123000205,24295,마천시장.마천동우방아파트앞,127.151510,37.499937,서울특별시,송파구,마천동,378.0
31561,100100228,3416,62,123000204,24294,마천동금호어울림1차아파트,127.154075,37.497978,서울특별시,송파구,마천동,473.0
31862,100100228,3416,63,123000202,24292,송파보훈회관.금호어울림2차,127.155567,37.493992,서울특별시,송파구,마천동,239.0
32149,100100228,3416,64,123000199,24289,송파구립마천청소년센터,127.156552,37.491982,서울특별시,송파구,마천동,140.0


In [220]:
def check_distance(cached, bus_no):
    org_df = globals()['src_df']

    each_route_df = org_df[org_df['노선명'] == bus_no]

    sums = (each_route_df['거리'] == 0.0).sum()

    if sums == 0:
        print(f"bus={bus_no} is done.")
        return

    original_indices = each_route_df.index

    for i in range(len(original_indices) - 1):
        idx1 = original_indices[i]
        idx2 = original_indices[i+1]  

        if org_df.loc[idx1, '거리'] == 0.0:
            print(f"bus={bus_no},idx={i} ==> break")

            lat1 = org_df.loc[idx1, 'Y좌표']
            lon1 = org_df.loc[idx1, 'X좌표']
            lat2 = org_df.loc[idx2, 'Y좌표']
            lon2 = org_df.loc[idx2, 'X좌표']

            new_distance = calcDistanceByApi(lon1, lat1, lon2, lat2)

            cache_key = (lat1, lon1, lat2, lon2)
                
            is_numeric = isinstance(new_distance, (int, float))
    
            if is_numeric and not np.isnan(new_distance):
                cached[cache_key] = new_distance
                org_df.loc[idx1, '거리'] = new_distance

            return


In [230]:
for x in range(len(bus_list)):
    for i in range(50):
        check_distance(DISTANCE_CACHE_LOADED, bus_list[x])

bus=3426,idx=62 ==> break
bus=3426,idx=63 ==> break
bus=3426,idx=64 ==> break
bus=3426,idx=65 ==> break
bus=3426,idx=66 ==> break
bus=3426,idx=67 ==> break
bus=3426,idx=68 ==> break
bus=3426,idx=69 ==> break
bus=3426,idx=70 ==> break
bus=3426,idx=71 ==> break
bus=3426,idx=72 ==> break
bus=3426,idx=73 ==> break
bus=3426,idx=74 ==> break
bus=3426,idx=75 ==> break
bus=3426,idx=76 ==> break
bus=3426,idx=77 ==> break
bus=3426,idx=78 ==> break
bus=3426,idx=79 ==> break
bus=3426,idx=80 ==> break
bus=3426,idx=81 ==> break
bus=3426,idx=82 ==> break
bus=3426,idx=83 ==> break
bus=3426,idx=84 ==> break
bus=3426,idx=85 ==> break
bus=361,idx=59 ==> break
bus=361,idx=60 ==> break
bus=361,idx=61 ==> break
bus=361,idx=62 ==> break
bus=361,idx=63 ==> break
bus=361,idx=64 ==> break
bus=361,idx=65 ==> break
bus=361,idx=66 ==> break
bus=361,idx=67 ==> break
bus=361,idx=68 ==> break
bus=361,idx=69 ==> break
bus=361,idx=70 ==> break
bus=361,idx=71 ==> break
bus=361,idx=72 ==> break
bus=361,idx=73 ==> break
b

In [235]:
src_df[src_df['노선명'] == bus_list[0]]

,ROUTE_ID,노선명,순번,NODE_ID,ARS_ID,정류소명,X좌표,Y좌표,시,구,동,거리
143,100100063,402,1,123000354,24446,장지공영차고지,127.135886,37.480507,서울특별시,송파구,장지동,377.0
1022,100100063,402,2,123000356,24448,아이코리아,127.133400,37.480972,서울특별시,송파구,장지동,190.0
1605,100100063,402,3,123000372,24464,송파글마루도서관.버들어린이집,127.131555,37.481607,서울특별시,송파구,장지동,271.0
2327,100100063,402,4,123000375,24467,문현고등학교,127.129206,37.480709,서울특별시,송파구,장지동,450.0
3014,100100063,402,5,123000382,24475,장지역.가든파이브라이프동,127.124614,37.479072,서울특별시,송파구,문정동,795.0
...,...,...,...,...,...,...,...,...,...,...,...,...
39714,100100063,402,106,123000383,24476,장지역.가든파이브라이프동,127.124888,37.479007,서울특별시,송파구,문정동,446.0
39816,100100063,402,107,123000374,24466,문현고등학교.송파파인타운5단지,127.129475,37.480632,서울특별시,송파구,장지동,241.0
39891,100100063,402,108,123000373,24465,송파글마루도서관.버들어린이집,127.131337,37.481598,서울특별시,송파구,장지동,224.0
39979,100100063,402,109,123000379,24471,아이코리아앞,127.133640,37.480960,서울특별시,송파구,장지동,297.0


In [228]:
bus_list

['3426',
 '361',
 '402',
 '4425',
 '500',
 '504',
 '5525',
 '5535',
 '6001',
 '6005',
 '6006',
 '6007',
 '6008',
 '6009',
 '6013',
 '6016',
 '6017',
 '6019',
 '6020',
 '6103',
 '6200',
 '6300',
 '6600',
 '6703',
 '6705A',
 'N6703',
 '강남03',
 '강남06',
 '강남06-1',
 '강남06-2',
 '서초06',
 '서초16',
 '서초17']

In [234]:
bus_list.pop(0)

'361'

In [236]:
print(f"'특정컬럼'의 값이 0.0인 행의 개수: {(src_df['거리'] == 0.0).sum()-705}")

'특정컬럼'의 값이 0.0인 행의 개수: 0


In [237]:
import pickle

def save_cache_to_pickle(cache, filename="distance_cache.pkl"):
  with open(filename, 'wb') as f: # 'wb'는 Write Binary 모드
    pickle.dump(cache, f)
  print(f"캐시가 '{filename}'에 Pickle 형식으로 저장되었습니다.")

In [238]:
save_cache_to_pickle(DISTANCE_CACHE_LOADED, "20251027_DISTANCE_CACHE.pkl")

캐시가 '20251027_DISTANCE_CACHE.pkl'에 Pickle 형식으로 저장되었습니다.


In [239]:
print("DISTANCE_CACHE=",len(DISTANCE_CACHE_LOADED))

DISTANCE_CACHE= 17239


In [240]:
import math


def check_nan_in_dict(d):
    for value in d.values():
        # 값이 float 타입이고, math.isnan()을 통과하는지 확인
        if isinstance(value, float) and math.isnan(value):
            print()
            return True
    return False

result = check_nan_in_dict(DISTANCE_CACHE_LOADED)

print(f"DISTANCE_CACHE에 NaN이 있습니까?: {result}")

DISTANCE_CACHE에 NaN이 있습니까?: False


In [241]:
src_df.to_csv('20251027_서울시_버스정류장_주소_거리.csv', index=False, encoding='utf-8')